## Apply FA*IR post-processing re-ranking (inspired by Zehlike et al.) to improve group fairness in top-K recommendations

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# === Step 1: Load training interactions to calculate item frequency ===
train_inter_df = pd.read_csv('../datasets/split_datasets/ml-1m/ml-1m.train.inter', sep='\t')

# Count item frequency in training data
item_freq = train_inter_df['item_id:token'].value_counts()
item_freq.index = item_freq.index.astype(str)
print(f"Total unique items in training: {len(item_freq)}")

Total unique items in training: 3683


In [2]:
# Sort items by frequency (high to low)
item_freq_sorted = item_freq.sort_values(ascending=False)

# Define long-tail cutoff (e.g., bottom 20%)
tail_ratio = 0.2
tail_cutoff_index = int((1 - tail_ratio) * len(item_freq_sorted))
tail_item_ids = set(item_freq_sorted.index[tail_cutoff_index:])

print(f"Defined long-tail as bottom {tail_ratio*100:.0f}% of items")
print(f"Tail threshold frequency (≤): {item_freq_sorted.iloc[tail_cutoff_index]}")
print(f"Number of long-tail items: {len(tail_item_ids)}")

Defined long-tail as bottom 20% of items
Tail threshold frequency (≤): 19
Number of long-tail items: 737


In [3]:
# Load top-30 recommendation results ===
topk_df = pd.read_csv('outputs/ml_all_user_top30.csv')  # Format: user_id, topk_items
user_df = pd.read_csv('../datasets/atomic_datasets/ml-1m/ml-1m.user', sep='\t')
user2gender = dict(zip(user_df['user_id:token'], user_df['gender:token']))

print(f"Loaded Top-30 recommendations for {len(topk_df)} users")

Loaded Top-30 recommendations for 6040 users


In [4]:
K = 10  # Only evaluate the top-10
total_tail_before = 0

for items in topk_df['topk_items']:
    item_list = str(items).split(',')[:K]  # only use top-10
    tail_count = sum(i in tail_item_ids for i in item_list)
    total_tail_before += tail_count

num_users = len(topk_df)
avg_tail_items = total_tail_before / num_users
tail_ratio = avg_tail_items / K

print("\n=== Long-Tail Ratio in Top-10 ===")
print(f"Average long-tail items per user: {avg_tail_items:.4f}")
print(f"Long-tail item ratio in Top-10: {tail_ratio:.2%}")


=== Long-Tail Ratio in Top-10 ===
Average long-tail items per user: 0.0025
Long-tail item ratio in Top-10: 0.02%


In [5]:
# === Step 4: Apply FA*IR-like re-ranking to increase long-tail item exposure ===
desired_tail_ratio = 0.1  # e.g., at least 1 item in Top-10 are from the tail
reranked_result = []
modified_users = 0
total_tail_after = 0

def is_tail(item_id):
    return item_id in tail_item_ids

for _, row in topk_df.iterrows():
    user_id = row['user_id']
    top_items = str(row['topk_items']).split(',')
    top10 = top_items[:K]
    
    # Case 1: already contains tail item — keep as-is
    if any(is_tail(i) for i in top10):
        reranked = top10
    else:
        # Case 2: find first tail item in top-30
        tail_candidate = next((i for i in top_items if is_tail(i) and i not in top10), None)
        if tail_candidate:
            reranked = top10[:-1] + [tail_candidate]  # Replace 10th
            modified_users += 1
        else:
            reranked = top10  # no tail found in top-30, leave unchanged        

    tail_count = sum(1 for i in reranked if is_tail(i))
    total_tail_after += tail_count

    total_tail_after += sum(i in tail_item_ids for i in reranked)

    reranked_result.append({
        'user_id': user_id,
        'gender': user2gender.get(user_id, 'UNK'),
        'topk_items': ','.join(reranked)
    })


In [6]:
fair_top10_df = pd.DataFrame(reranked_result)
display(fair_top10_df.head())
avg_tail = total_tail_after / len(fair_top10_df)

print("\n=== Minimal FA*IR Re-ranking Summary ===")
print(f"Total users processed: {len(fair_top10_df)}")
print(f"Users modified (tail added): {modified_users}")
print(f"Average long-tail items in Top-10: {avg_tail:.4f}")
print(f"Long-tail ratio in Top-10: {avg_tail / K:.2%}")

,user_id,gender,topk_items
0,1,F,"595,34,364,919,1,588,2081,3114,318,594"
1,2,M,"2028,1183,590,3418,527,1610,318,349,608,1393"
2,3,M,"1210,260,1196,1198,1270,480,356,1580,2716,110"
3,4,M,"1196,260,1210,1198,858,1214,1240,480,541,2028"
4,5,M,"2997,2692,2908,2858,2333,2599,2959,223,2395,2318"



=== Minimal FA*IR Re-ranking Summary ===
Total users processed: 6040
Users modified (tail added): 5
Average long-tail items in Top-10: 0.0066
Long-tail ratio in Top-10: 0.07%


In [7]:
# === Step 5: Save and report results ===
fair_top10_df.to_csv('outputs/ml_all_user_top10_fair.csv', index=False)
print("Saved re-ranked results to: outputs/ml_all_user_top10_fair.csv")

Saved re-ranked results to: outputs/ml_all_user_top10_fair.csv


# Apply post-processing calibration (inspired by Steck)to improve fairness in recommendation exposure